In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import glob

In [2]:
FileList = glob.glob('brisbane bar tide data/*.csv')
tide = pd.DataFrame()
for file in FileList:
    df = pd.read_csv(file, engine='python', header=None)
    if not tide.empty:
        tide = pd.concat([tide,df])
    else:
        tide = df
tide

,0
0,010120010000 1.530
1,010120010010 1.560
2,010120010020 1.590
3,010120010030 1.610
4,010120010040 1.630
...,...
52699,311220202310 1.750
52700,311220202320 1.720
52701,311220202330 1.680
52702,311220202340 1.640


In [3]:
tide.rename(columns = {0:'data'}, inplace=True)
tide[['Time','Tide']] = tide.data.str.split(expand=True)
tide.drop(columns='data', inplace=True)
tide['Time'] = pd.to_datetime(tide['Time'], format='%d%m%Y%H%M')
tide = tide.sort_values('Time')
tide['Date'] = tide['Time'].dt.date
tide['Year'] = tide['Time'].dt.year
tide['Month'] = tide['Time'].dt.month
tide['Day'] = tide['Time'].dt.day

tide

,Time,Tide,Date,Year,Month,Day
0,2001-01-01 00:00:00,1.530,2001-01-01,2001,1,1
1,2001-01-01 00:10:00,1.560,2001-01-01,2001,1,1
2,2001-01-01 00:20:00,1.590,2001-01-01,2001,1,1
3,2001-01-01 00:30:00,1.610,2001-01-01,2001,1,1
4,2001-01-01 00:40:00,1.630,2001-01-01,2001,1,1
...,...,...,...,...,...,...
52699,2020-12-31 23:10:00,1.750,2020-12-31,2020,12,31
52700,2020-12-31 23:20:00,1.720,2020-12-31,2020,12,31
52701,2020-12-31 23:30:00,1.680,2020-12-31,2020,12,31
52702,2020-12-31 23:40:00,1.640,2020-12-31,2020,12,31


In [9]:
maxtide = list(tide.groupby(['Year','Month'])['Tide'].max().values)
mintide = list(tide.groupby(['Year','Month'])['Tide'].min().values)
#date = tide['Date'].unique()
date = pd.date_range('2001-1-1', '2020-12-31', freq='MS').to_pydatetime().tolist()
finaltide = pd.DataFrame({'Date':date, 'MaxTide':maxtide, 'MinTide':mintide})
finaltide['Date'] = pd.to_datetime(finaltide['Date'])
finaltide['Year'] = finaltide['Date'].dt.year
finaltide['Month'] = finaltide['Date'].dt.month
#finaltide['Day'] = finaltide['Date'].dt.day
finaltide.drop(columns='Date', inplace=True)
finaltide

,MaxTide,MinTide,Year,Month
0,2.780,0.100,2001,1
1,2.615,0.089,2001,2
2,2.874,0.276,2001,3
3,2.587,0.194,2001,4
4,2.603,0.218,2001,5
...,...,...,...,...
235,2.630,0.220,2020,8
236,2.590,0.190,2020,9
237,2.480,0.140,2020,10
238,2.610,0.150,2020,11


In [10]:
data = pd.read_csv('Mosquito_Data_2010_2020.csv')
columns = ['SITE_CODE', 'SURVEY_DATE', 'SURVEY_AREA_SQM', 'SURVEY_HABITAT', 'LARVAL_DENSITY_AV', 'TREATMENT_AREA_SQM', 'Qty (kg)', 'Rate']
df = data[columns]
df

C:\Users\yt335\Anaconda3\envs\tensorflow1.14\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (2,36,37,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SITE_CODE,SURVEY_DATE,SURVEY_AREA_SQM,SURVEY_HABITAT,LARVAL_DENSITY_AV,TREATMENT_AREA_SQM,Qty (kg),Rate
0,MCIS-2,3/07/2019,1000,Fresh,5,1000,0.3000,3.0
1,MCIS-3,3/07/2019,2200,Salt,10,2200,0.6600,3.0
2,MCIS-22,3/07/2019,1800,Fresh,NaN,1800,0.5400,3.0
3,MCIS-18,3/07/2019,1000,Salt,NaN,NaN,0.0000,NaN
4,MCIS-23,3/07/2019,400,Salt,NaN,NaN,0.0000,NaN
...,...,...,...,...,...,...,...,...
19644,RB24-1,29/05/2020,1200,Salt,NaN,NaN,0.00,NaN
19645,RB24-2,29/05/2020,350,Salt,NaN,NaN,0.00,NaN
19646,RB24-3,29/05/2020,400,Salt,NaN,NaN,0.00,NaN
19647,RB24-4,29/05/2020,2000,Salt,NaN,NaN,0.00,NaN


In [11]:
df['SITE_CODE'] = df['SITE_CODE'].str.upper() 
df['SITE_CODE'] = df['SITE_CODE'].str.replace('.','-')
df['SURVEY_HABITAT'] = df['SURVEY_HABITAT'].str.upper()
df = df.replace(' ', 0)
df = df.replace('<1', 0.5)
df = df.replace(df.iloc[5434,4], 0)
#df['LARVAL_DENSITY_AV'] = df['LARVAL_DENSITY_AV'].astype(int)
df['LARVAL_DENSITY_AV'] = pd.to_numeric(df['LARVAL_DENSITY_AV'])

C:\Users\yt335\Anaconda3\envs\tensorflow1.14\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\yt335\Anaconda3\envs\tensorflow1.14\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yt335\Anaconda3\envs\tensorflow1.14\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [12]:
df.loc[df['SURVEY_HABITAT'] == 'SALT', 'SURVEY_HABITAT'] = 7
df.loc[df['SURVEY_HABITAT'] == 'FRESH', 'SURVEY_HABITAT'] = 4
df.loc[df['SURVEY_HABITAT'] == 'DRY', 'SURVEY_HABITAT'] = 1
df.loc[df['SURVEY_HABITAT'] == 'DRAIN', 'SURVEY_HABITAT'] = 5
df.loc[df['SURVEY_HABITAT'] == 'BRACKISH', 'SURVEY_HABITAT'] = 6
df.loc[df['SURVEY_HABITAT'] == 'SWAMP', 'SURVEY_HABITAT'] = 3
df.loc[df['SURVEY_HABITAT'] == 'DAM', 'SURVEY_HABITAT'] = 2
df.loc[df['SURVEY_HABITAT'] == 'TIDAL', 'SURVEY_HABITAT'] = 0
df.loc[df['SURVEY_HABITAT'] == 'ROUTINE', 'SURVEY_HABITAT'] = 0
df.fillna(value=0, inplace=True)

In [13]:
df['SURVEY_DATE'] = pd.to_datetime(df['SURVEY_DATE'], format='%d/%m/%Y')
df['Year'] = df['SURVEY_DATE'].dt.strftime('%Y')
df['Month'] = df['SURVEY_DATE'].dt.strftime('%m')
df['Day'] = df['SURVEY_DATE'].dt.strftime('%d')
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Day'] = df['Day'].astype(int)

In [16]:
longalti = pd.read_excel('target_site.xlsx')
longalti

,OBJECTID,GIS ID,Description,Suburb,Survey Type,Ownership,Zone ID,Map Code,Site Code,Water Type,Status ID,Fish Habitat,Marine Park,Marine Zone,Marine Category,Status,Label Latitude,Label Longitude,Shape_Length,Shape_Area
0,17,802,Pannikin Island,20,1,3,3,PAIS,1,17,3,0,1,2,2,Aerial,-27.647267,153.330172,5084.016790,9.454191e+05
1,21,804,Long Island,24,1,3,3,LOIS,1,17,3,0,1,2,2,Aerial,-20.361276,148.854876,4845.830661,9.625326e+05
2,37,107,Giants Grave,25,1,1,4,RUIS,5,17,3,0,1,2,3,Aerial,-27.670555,153.368057,2565.552368,2.778377e+05
3,54,703,Scout Centre,30,1,1,2,RB13,5,18,3,0,1,2,1,Aerial,-27.441991,153.001805,2042.489982,1.164107e+05
4,57,168,Golf Course Lot 9 RP 30558 foreshore north,24,1,1,2,RB24,3,17,5,0,0,0,1,Ground,-27.596192,153.297857,79.460251,3.588112e+02
5,59,166,Golf Course Lot 10 RP 30558 foreshore east,24,1,1,2,RB24,1,17,5,0,0,0,3,Ground,-27.599444,153.299574,151.472962,1.040940e+03
6,60,167,Golf Course Lot 10 RP 30558 foreshore west,24,1,1,2,RB24,2,17,5,0,0,0,3,Ground,-27.600034,153.294274,126.140393,3.708127e+02
7,61,173,Golf Course via North St 4th site,24,1,1,2,RB21,3,17,4,0,0,0,3,Ground,-27.602924,153.298415,118.205684,1.039065e+02
8,62,59,Horse Farm,24,1,1,2,G12,2,17,4,0,0,0,0,Aerial,-27.701886,157.298197,1899.121030,1.075416e+05
9,63,3,Prawn Farm,24,1,1,2,W44,1,17,3,0,0,0,1,Aerial,-27.697614,153.315753,3327.127335,1.815469e+05


In [17]:
longalti['Site Code'] = longalti['Site Code'].astype(str)
longalti['SITE_CODE'] = longalti[['Map Code', 'Site Code']].agg('-'.join, axis=1)
columns_longalti = ['SITE_CODE','Label Latitude','Label Longitude']
df_longalti = longalti[columns_longalti]

In [18]:
temperature = pd.read_csv('IDCJAC0002_040913_Data1.csv')
temperature = temperature[['Year','Month','Mean maximum temperature (°C)']]
temperature

,Year,Month,Mean maximum temperature (°C)
0,1999,12,27.2
1,2000,1,29.3
2,2000,2,28.4
3,2000,3,28.8
4,2000,4,26.5
...,...,...,...
242,2020,2,29.9
243,2020,3,28.7
244,2020,4,28.7
245,2020,5,24.0


In [19]:
rainfall = pd.read_csv('IDCJAC0001_040913_Data1.csv')
rainfall = rainfall[['Year','Month', 'Monthly Precipitation Total (millimetres)']]
rainfall.fillna(value=0, inplace=True)
rainfall

,Year,Month,Monthly Precipitation Total (millimetres)
0,1999,12,161.0
1,2000,1,40.4
2,2000,2,65.0
3,2000,3,83.8
4,2000,4,42.8
...,...,...,...
237,2020,1,165.6
238,2020,2,365.8
239,2020,4,5.2
240,2020,5,12.2


In [20]:
target_site = df.groupby('SITE_CODE').count().sort_values('SURVEY_DATE')[df.groupby('SITE_CODE').count().sort_values('SURVEY_DATE')['SURVEY_DATE']>=150].index
df2 = df[df['SITE_CODE'].isin(target_site)]
df2 = df2.sort_values('SURVEY_DATE')
df2.reset_index(drop = True, inplace = True)

In [21]:
dff = pd.merge(df2, df_longalti)
dff = pd.merge(dff, temperature, on=['Year','Month'], how='left')
dff = pd.merge(dff, rainfall, on=['Year','Month'], how='left')
dff['Monthly Precipitation Total (millimetres)'].fillna(value=0, inplace=True)
dff

,SITE_CODE,SURVEY_DATE,SURVEY_AREA_SQM,SURVEY_HABITAT,LARVAL_DENSITY_AV,TREATMENT_AREA_SQM,Qty (kg),Rate,Year,Month,Day,Label Latitude,Label Longitude,Mean maximum temperature (°C),Monthly Precipitation Total (millimetres)
0,RB21-5,2010-01-04,100,5,0.0,0,0,0,2010,1,4,-27.597295,153.296119,31.1,40.2
1,RB21-5,2010-01-18,100,5,0.0,0,0,0,2010,1,18,-27.597295,153.296119,31.1,40.2
2,RB21-5,2010-01-21,100,5,0.0,0,0,0,2010,1,21,-27.597295,153.296119,31.1,40.2
3,RB21-5,2010-02-03,100,5,60.0,100,0.077,7,2010,2,3,-27.597295,153.296119,30.1,272.4
4,RB21-5,2010-02-12,100,5,9.0,0,0,0,2010,2,12,-27.597295,153.296119,30.1,272.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,RB14-12,2020-04-14,52547,7,6.0,100,0.03,3.0,2020,4,14,-27.565339,153.282744,28.7,5.2
8052,RB14-12,2020-05-07,52547,7,0.0,0,0.00,0,2020,5,7,-27.565339,153.282744,24.0,12.2
8053,RB14-12,2020-05-08,10000,7,0.0,0,0.00,0,2020,5,8,-27.565339,153.282744,24.0,12.2
8054,RB14-12,2020-05-25,52547,7,0.0,0,0.00,0,2020,5,25,-27.565339,153.282744,24.0,12.2


In [22]:
#無風險=0, 低風險<9, 中風險<20, 高風險>20
#np.percentile(list(dff[dff['LARVAL_DENSITY_AV']!=0]['LARVAL_DENSITY_AV']), [33, 66])
#plt.bar(range(len(dff['LARVAL_DENSITY_AV'].sort_values())), dff['LARVAL_DENSITY_AV'].sort_values())
dff.loc[dff['LARVAL_DENSITY_AV']==0, 'risk'] = 0
dff.loc[(dff['LARVAL_DENSITY_AV']<=9)&(dff['LARVAL_DENSITY_AV']>0), 'risk'] = 1
dff.loc[(dff['LARVAL_DENSITY_AV']<=20)&(dff['LARVAL_DENSITY_AV']>9), 'risk'] = 2
dff.loc[dff['LARVAL_DENSITY_AV']>20, 'risk'] = 3
dff

,SITE_CODE,SURVEY_DATE,SURVEY_AREA_SQM,SURVEY_HABITAT,LARVAL_DENSITY_AV,TREATMENT_AREA_SQM,Qty (kg),Rate,Year,Month,Day,Label Latitude,Label Longitude,Mean maximum temperature (°C),Monthly Precipitation Total (millimetres),risk
0,RB21-5,2010-01-04,100,5,0.0,0,0,0,2010,1,4,-27.597295,153.296119,31.1,40.2,0.0
1,RB21-5,2010-01-18,100,5,0.0,0,0,0,2010,1,18,-27.597295,153.296119,31.1,40.2,0.0
2,RB21-5,2010-01-21,100,5,0.0,0,0,0,2010,1,21,-27.597295,153.296119,31.1,40.2,0.0
3,RB21-5,2010-02-03,100,5,60.0,100,0.077,7,2010,2,3,-27.597295,153.296119,30.1,272.4,3.0
4,RB21-5,2010-02-12,100,5,9.0,0,0,0,2010,2,12,-27.597295,153.296119,30.1,272.4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,RB14-12,2020-04-14,52547,7,6.0,100,0.03,3.0,2020,4,14,-27.565339,153.282744,28.7,5.2,1.0
8052,RB14-12,2020-05-07,52547,7,0.0,0,0.00,0,2020,5,7,-27.565339,153.282744,24.0,12.2,0.0
8053,RB14-12,2020-05-08,10000,7,0.0,0,0.00,0,2020,5,8,-27.565339,153.282744,24.0,12.2,0.0
8054,RB14-12,2020-05-25,52547,7,0.0,0,0.00,0,2020,5,25,-27.565339,153.282744,24.0,12.2,0.0


In [23]:
dff.isnull().any()

SITE_CODE                                    False
SURVEY_DATE                                  False
SURVEY_AREA_SQM                              False
SURVEY_HABITAT                               False
LARVAL_DENSITY_AV                            False
TREATMENT_AREA_SQM                           False
Qty (kg)                                     False
Rate                                         False
Year                                         False
Month                                        False
Day                                          False
Label Latitude                               False
Label Longitude                              False
Mean maximum temperature (°C)                False
Monthly Precipitation Total (millimetres)    False
risk                                         False
dtype: bool